## Wordle Game Clone

This code aims to create a wordle clone

#### Step 1 Library & Data Load 

In [48]:
import random

def uppercase_sort_wordlist(unedited_file, edited_file, copy_edited_file):
    # Read the unedited wordlist
    with open(unedited_file, 'r') as file:
        words = file.read().splitlines()
        uppercase_words = [word.upper() for word in words]
        combined_words = set(uppercase_words)
        sorted_words = sorted(combined_words)

    with open(edited_file, 'w') as file:
        file.write('\n'.join(sorted_words))
    
    with open(copy_edited_file, 'w') as file:
        file.write('\n'.join(sorted_words))
        
  #  return sorted_words


#### Step 2 Word bank list to set
The other text files have some words that may be associated with another language other than english or they have punctuation or they are proper names like cities. These are not usually in wordle games so I have left them 

In [49]:
#word_bank = set(florida_words) | set(github_words) | set(sgb_words) | set(combined_words) # Convert to a set for faster lookups

#word_bank = uppercase_sort_wordlist('combined_wordle.txt', 'wordle_words_sorted.txt')

uppercase_sort_wordlist('combined_wordlist.txt', 'wordle_words_sorted.txt', 'cap_wordle_check_words.txt')
#print(list(wordle_words)[:10])



#### Step 3 Game Instruction Function

In [5]:
def Game_Instruction():
    print(""" 
    Wordle is a single player word guessing game. 
    The player has six attempts to guess a five-letter word.
    After each guess, the player receives feedback on the letters in their guess. The feedback is as follows:
    "☑" indicates a letter is in the correct position,
    "☐" indicates a letter is in the word but in the wrong position,
    "☒" indicates a letter is not in the word.
    The player must use the feedback to make educated guesses and narrow down the possibilities.
    Good Luck!
          """)

#### Step 4 Function to choose word from word bank 
This code defines a class called WordPicker which choses a random word from the word bank. It is designed so that when a user plays again it will pick a different word from the list so that the same word isnt selected in a row. After all words are selected, the word bank is reset so that all the words EXCEPT the word that was last chosen are possible options to be the chosen word. This allows replayability.

Making this process into a class also allows for any set or list of words to be used as the word bank in the game. It also makes the option of playing again easier.

In [ ]:
class WordPicker:
    def __init__(self, combined_wordlist_path, chosen_wordlist_path, check_wordlist_path):
        """
        Initializes the WordPicker with file paths for the word lists.
        """
        self.combined_wordlist_path = combined_wordlist_path
        self.chosen_wordlist_path = chosen_wordlist_path
        self.last_word = None
        self.check_wordlist_path = check_wordlist_path

        
        # ensure files exist         
        open(self.combined_wordlist_path, 'a').close()
        open(self.chosen_wordlist_path, 'a').close()
        open(self.check_wordlist_path, 'a').close()
        
    def load_wordlist(self, file_path):
        """
        Loads a word list from variable made at loading.
        """
        with open(file_path, 'r') as file:
            return set(file.read().splitlines())
        
    def save_wordlist(self, file_path, wordlist):
        """
        Saves a word list to the specified file path.
        """
        with open(file_path, 'w') as file:
            file.write('\n'.join(wordlist))
            
    def pick_word(self):
        """
        Picks a word from combined_wordlist.txt, moves it to chosen_wordle.txt,
        and ensures the same word is not chosen consecutively.
        """
        # Load words from the files
        combined_words = self.load_wordlist(self.combined_wordlist_path)
        chosen_words = self.load_wordlist(self.chosen_wordlist_path)

        # If combined_wordlist.txt is empty, refill it from chosen_wordle.txt
        if not combined_words:
            combined_words = chosen_words
            chosen_words = set()
            self.save_wordlist(self.combined_wordlist_path, combined_words)
            self.save_wordlist(self.chosen_wordlist_path, chosen_words)

        # Convert to a list for shuffling
        combined_words = list(combined_words)
        random.shuffle(combined_words)

        # Pick a word that is not the same as the last word
        while combined_words:
            chosen_word = combined_words.pop()
            if chosen_word != self.last_word or not combined_words:
                break

        # Update last_word and move the chosen word to chosen_wordle.txt
        self.last_word = chosen_word
        chosen_words.add(chosen_word)
        self.save_wordlist(self.chosen_wordlist_path, chosen_words)

        # Remove the chosen word from combined_wordlist.txt
        combined_words = set(combined_words)
        self.save_wordlist(self.combined_wordlist_path, combined_words)

        print(f"\n Chosen word: {chosen_word}")
        return chosen_word

In [17]:
words = {"slate", "stale", "panel"}
class WordPicker:
    def __init__(self, Word_set):
       
        self.all_words = list(Word_set)
        self.all_words = [word.upper() for word in self.all_words]
        print(self.all_words)
        self.last_word = None
        self.reset_pool()
    
    def reset_pool(self):
        self.remaining_words = self.all_words.copy()
        random.shuffle(self.remaining_words)
        
        # last word isnt the first chosen word in the new shuffle
        if self.last_word in self.remaining_words and len(self.remaining_words) > 1:
            while self.remaining_words[0] == self.last_word:
                random.shuffle(self.remaining_words)
           
        
    def pick_word(self):
        if not self.remaining_words:
            self.reset_pool()
            
        while True:
            print(f"Remaining words: {self.remaining_words}")
            chosen_word = self.remaining_words.pop()
            if chosen_word != self.last_word or not self.remaining_words:
                break
            else:
                # put word back in pool and shuffle again to avoid back to back repeats
                self.remaining_words.insert(0, chosen_word)
                #print("reshuffle")
                random.shuffle(self.remaining_words)
                
        self.last_word = chosen_word.upper()
        
        print(f"Chosen word: {chosen_word}\n")
        
        return chosen_word
    
picker = WordPicker(words)
picker.pick_word()


['PANEL', 'STALE', 'SLATE']
Remaining words: ['STALE', 'PANEL', 'SLATE']
Chosen word: SLATE



'SLATE'

#### Step 5 Function loop to check user inputed word against chosen word

In [ ]:
def check_word():
    picker = WordPicker('wordle_words_sorted.txt', 'chosen_wordle.txt', 'cap_wordle_check_words.txt')
    
    chosen_word = picker.pick_word()
    attempt = 1
    max_attempt = 6
    revealed_positions = ["☒"] * len(chosen_word) 

    
    while attempt <= max_attempt:
        print(f"Attempt {attempt}/{max_attempt}")
        print("Enter Guess or type 'HINT' for a hint or 'END' to end the game.\n")
        guess = str(input("Enter your guess: \n")).upper().strip()
        
        ## check for user end
        if guess == "END":
            print("Ending game session.")
            break
        
        ## check for hint request
        elif guess == "HINT":   
            provide_hint(chosen_word, revealed_positions) # type: ignore
            continue
        
        ## check if word is 5 letters
        if len(guess) != 5:
            print("Word must be 5 letters, try again. \n")
            continue
         
        ## check if word is in word bank  
        elif guess not in picker.load_wordlist('cap_wordle_check_words.txt'):
            print(f"{guess} is not in word dictionary, try again. \n")
            continue
        
        ## check if guess is the same as chosen word
        if guess == chosen_word:
            print(f"Congratulations! You guessed the word, {guess}, in {attempt} attempts!\n")
            replay_game()
            return
        else:
            attempt += 1
            result = []
            chosen_word_counts = {}
            print(f"User Guess: {guess}")
            
            # count occurrences of each letter in the chosen word
            for letter in chosen_word:
                chosen_word_counts[letter] = chosen_word_counts.get(letter, 0) + 1
            
            ## first check for correct letters in the correct position
            for i, letter in enumerate(guess):
                if letter == chosen_word[i]:
                    result.append(f"{letter}☑")
                    chosen_word_counts[letter] -= 1
                    revealed_positions[i] = "☑"  # mark the position as revealed
                else:
                    result.append(None)

            ## second check for correct letters in the wrong position
            for i, letter in enumerate(guess):
                if result[i] is None:  # only check letters that are not already marked as correct
                    if letter in chosen_word_counts and chosen_word_counts[letter] > 0:
                        result[i] = f"{letter}☐"
                        chosen_word_counts[letter] -= 1
                        revealed_positions[i] = "☐" # mark the position as correct but wrong position
                        
                    else:
                        result[i] = f"{letter}☒"
                        revealed_positions[i] = "☒"
            
            print(" ".join(result))
            #print(f"Revealed positions: {revealed_positions}")
            if attempt > max_attempt:
                print(f"Sorry, you've used all your attempts. The word was: {chosen_word}\n")
                replay_game()
                return
                
                        
       
check_word()

Chosen word: YOMPS
Attempt 1/6
Enter Guess or type 'HINT' for a hint or 'END' to end the game.

User Guess: STORM
S☐ T☒ O☐ R☒ M☐
Revealed positions: ['☐', '☒', '☐', '☒', '☐']
Attempt 2/6
Enter Guess or type 'HINT' for a hint or 'END' to end the game.

MOORE is not in word dictionary, try again. 

Attempt 2/6
Enter Guess or type 'HINT' for a hint or 'END' to end the game.

User Guess: MUSHY
M☐ U☒ S☐ H☒ Y☐
Revealed positions: ['☐', '☒', '☐', '☒', '☐']
Attempt 3/6
Enter Guess or type 'HINT' for a hint or 'END' to end the game.

OPOID is not in word dictionary, try again. 

Attempt 3/6
Enter Guess or type 'HINT' for a hint or 'END' to end the game.

Word must be 5 letters, try again. 

Attempt 3/6
Enter Guess or type 'HINT' for a hint or 'END' to end the game.

User Guess: FOUND
F☒ O☑ U☒ N☒ D☒
Revealed positions: ['☒', '☑', '☒', '☒', '☒']
Attempt 4/6
Enter Guess or type 'HINT' for a hint or 'END' to end the game.

COSMY is not in word dictionary, try again. 

Attempt 4/6
Enter Guess or typ

#### Step 6 Function to keep track of how many attempts it takes to guess correct word. Show distribution of guessing success. 

#### Step 6A Function for user input to replay game

In [ ]:
def replay_game():
    """
    Asks the user if they want to play again. If yes, it restarts the game. If no, it exits the program.
    """
    while True:
        print("\nDo you want to play again? (yes/no)\n")
        play_again = input("Do you want to play again? (yes/no): ").strip().lower()
        if play_again == "yes":
            print("Starting a new game...\n")
            check_word()
            break
        elif play_again == "no":
            print("Thanks for playing!")
            break
        else:
            print("Invalid input. Please enter 'yes' or 'no'.")
            continue

#### Step 7 Hint function
Since I know I have a few words in the word bank that are obscure, I want to include a hint function. The hint function should be accessed when the user enters the word "hint" as a guess, it will not count as an attempt and will instead access an option menu of possible hint types the user can chose from. The user only gets one hint per chosen word. The different hints the user can chose from are

- asking if there are any double letters
- fill in letter 1
- fill in letter 2
- fill in letter 3
- fill in letter 4
- fill in letter 5

If there is only 1 incorrect letter remaining "(all 4 other letters in the word are marked ☑)" then the fill in letter options are not available, only the double letter question will be displayed. 

If a hint is called by a user the success counter will not increase for a correct guess for that chosen word.

In [9]:
def provide_hint(chosen_word, revealed_positions):
    """
    Provides a hint to the user based on their choice.
    :param chosen_word: The word to guess.
    :param revealed_positions: A list indicating which positions are already revealed.
    """
    
    while True:
        ## special case for 4 correct letters 
        if revealed_positions.count("☑") == 4:
            print("Hint Options:")
            print("6. Are there any double letters?")
            print("7. Cancel hint\n")
            try:
                choice = int(input("Choose a hint option (6 or 7): ").strip())
                if choice not in (6, 7):
                    raise ValueError("Invalid input. Please enter 6 or 7.")
            except ValueError:
                print("Invalid input. Please enter 6 or 7.") 
                continue
        elif revealed_positions.count("☑") == 3 and revealed_positions.count("☐") == 2:
            print("Hint not available. Ending Hint session.\n")
            break
        else:
            print("Hint Options:")
            print("1. Reveal letter 1")
            print("2. Reveal letter 2")
            print("3. Reveal letter 3")
            print("4. Reveal letter 4")
            print("5. Reveal letter 5")
            print("6. Are there any double letters?")
            print("7. Cancel hint\n")
            
            try:
                choice = int(input("Enter the number of your choice: ").strip())
            except ValueError:
                print("Invalid input. Please enter a number.")
                continue
        
        ## double letter check    
        if choice == 6:
            # Check for double letters
            has_double_letters = any(chosen_word.count(char) > 1 for char in set(chosen_word))
            if has_double_letters:
                print("Yes, there are double letters in the word.\n")
            else:
                print("No, there are no double letters in the word.\n")
            break       
        ## letter reveal
        elif 1 <= choice <= 5:
                # Reveal a specific letter
                position = choice - 1  # Adjust for 0-based index
                letnum = choice
                #print(f"full list {revealed_positions}, letter number and letter: {letnum};{revealed_positions[position]}")
                
                if revealed_positions[position] == "☑":
                    print(f"Letter {letnum} is already revealed: '{chosen_word[position]}'. Please select another hint option.\n")
                    continue
                
                else:
                    print(f"Letter {letnum} is: {chosen_word[position]}\n")
                    
                break
         
        elif choice == 7:
            print("Hint cancelled.")
            break
        else:
                print("Invalid choice. Please select a valid option.")

